# AgriSmart

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

from sklearn.preprocessing import LabelEncoder
import joblib


In [2]:
# Data path (change it if needed)
data_dir = "./PlantVillage/"  

In [3]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [4]:
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

Found 33027 images belonging to 16 classes.


In [5]:
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 8249 images belonging to 16 classes.


In [6]:
# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(train_data.num_classes, activation="softmax")
])


c:\Users\Akshat\PROJECTS\AgriSmart\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [8]:
# Train model
# model.fit(train_data, validation_data=val_data, epochs=10)

In [9]:

# Save model
model.save("disease_model.h5")
print("✅ Model saved as disease_model.h5")

# 1. ImageDataGenerator से क्लास के नाम प्राप्त करें
class_names = list(train_data.class_indices.keys())

# 2. इन नामों के लिए एक नया LabelEncoder बनाएं और फिट करें
disease_label_encoder = LabelEncoder()
disease_label_encoder.fit(class_names)

# 3. इस नए एन्कोडर को एक अलग फ़ाइल में सहेजें
joblib.dump(disease_label_encoder, 'disease_label_encoder.pkl')

print(f"\n✅ Disease labels found: {class_names}")
print("✅ Encoder saved successfully as disease_label_encoder.pkl")

✅ Model saved as disease_model.h5

✅ Disease labels found: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'PlantVillage', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
✅ Encoder saved successfully as disease_label_encoder.pkl
